## MS-SMC' validation

This notebook validates the MS-SMC' calculations from McKenzie and Eaton (XXXX) by comparing the analytical predictions to the results of stochastic coalescent simulations.

In [1]:
# conda install ipcoal toytree -c conda-forge

In [2]:
from concurrent.futures import ProcessPoolExecutor
import numpy as np
import pandas as pd
import ipcoal
import toyplot, toyplot.svg, toyplot.png
import toytree
from scipy import stats

### Parameters
These are global parameters used for analyses in the manuscript. The NLOCI variable is changed further below.

In [13]:
RECOMB = 2e-9
SPECIES_TREE_HEIGHT = 1e6
NEFF_MIN = 50_000
NEFF_MAX = 500_000
NEFF_VALS = 10
SEED = 123
NLOCI = 4
NREPS = 10

### Generate a species tree for model 1 tip, 2 tips, or 8 tips
There are the 3 species tree models used. Below is a function for generating them.

In [4]:
sptree = toytree.tree("(a);")
sptree = sptree.mod.edges_scale_to_root_height(treeheight=SPECIES_TREE_HEIGHT, include_stem=True)
sptree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="255.0px" viewBox="0 0 350.0 255.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t50634c940bd84b1faed8d0c32521c14a"> 0 1 a 0 250000 500000

In [5]:
sptree = toytree.rtree.baltree(2)
sptree = sptree.mod.edges_scale_to_root_height(treeheight=SPECIES_TREE_HEIGHT, include_stem=True)
sptree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc146943925984e52ae85bd24858c230e"> 0 1 2 r0 r1 0 250000 500000

In [6]:
sptree = toytree.rtree.baltree(8)
sptree = sptree.mod.edges_scale_to_root_height(treeheight=SPECIES_TREE_HEIGHT, include_stem=True)
sptree.draw('p');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="260.0px" viewBox="0 0 350.0 260.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tdd122d1b7b2f4d7a9d3f7239b54b323c"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 r0 r1 r2 r3 r4 r5 r6 r7 0 250000 500000 750000

In [7]:
def get_sptree(ntips: int) -> toytree.ToyTree:
    """Return a balanced species tree with the specified number of tips and even branch lengths."""
    if ntips == 1:
        sptree = toytree.tree("(a);")
    else:
        sptree = toytree.rtree.baltree(ntips)
    sptree = sptree.mod.edges_scale_to_root_height(SPECIES_TREE_HEIGHT, include_stem=True)
    return sptree

### Get waiting distances to event x:

This is the main function for validating predictions against simulations.

In [8]:
def get_sim_and_analytical(
    sptree: toytree.ToyTree, nsamples: int, neff: int, nsites: int, seed: int, smc: bool,
    ) -> np.ndarray:
    """Return the prob and waiting distance for tree-change and topo-change for analytical and 
    simulations.
    
    Analytical results are computed for the first genealogy in a locus. Simulated results 
    are found by examining subsequent genealogies in the tree sequence until each event
    type is observed. 
    
    This uses some global variables (see above).
    
    Parameters
    ----------
    sptree: ToyTree
        A species tree with edge lengths in units of generations
    nsamples: int
        Number of haploid sampler per species tree tip.
    neff: int
        A diploid effective population size applied to all sptree edges.
    nsites: int
        Number of linked sites to simulate.
    seed: int
        A seed for the random number generator
    """
    model = ipcoal.Model(
        sptree, 
        Ne=neff, 
        seed_trees=seed, 
        nsamples=nsamples, 
        recomb=RECOMB,
        record_full_arg=True,
        ancestry_model="smc_prime" if smc else "hudson",  # use default (Hudson)
    )
    imap = model.get_imap_dict()
    
    # store results in arrays
    # 0 = smc_tree_probs
    # 1 = smc_topo_probs
    # 2 = smc_tree_dists
    # 3 = smc_topo_dists
    # 4 = sim_tree_dists
    # 5 = sim_topo_dists
    # 6 = sum_edge_lengths of genealogy 0
    # 7 = event type
    results = np.zeros(shape=(NLOCI, 8))
    
    # iterate over number of loci (replicates)
    for lidx in range(NLOCI):
        
        # get FULL tree sequence given the parameterized ipcoal.Model
        ts = next(model._get_tree_sequence_generator(nsites=nsites))
        
        # get SIMPLE tree sequence for fetching simplified trees at breakpoints
        # from the FULL `ts`, since no-change breakpoints disappear in `sts`.
        sts = ts.simplify(filter_sites=False)

        # get the starting tree and its sum branch lengths
        tree0 = sts.at(0, sample_lists=True)
        tsumlen0 = tree0.get_total_branch_length()
        results[lidx, 6] = tsumlen0

        # compute analytical probabilities of change given tree0
        toy0 = toytree.tree(tree0.as_newick(node_labels=model.tipdict))
        results[lidx, 0] = ipcoal.smc.get_probability_of_tree_change(model.tree, toy0, imap)
        results[lidx, 1] = ipcoal.smc.get_probability_of_topology_change(model.tree, toy0, imap)
        
        # compute analytical E[waiting distances] given tree0
        tree_rate = tsumlen0 * results[lidx, 0] * RECOMB
        topo_rate = tsumlen0 * results[lidx, 1] * RECOMB
        results[lidx, 2] = stats.expon.mean(scale=1 / tree_rate)
        results[lidx, 3] = stats.expon.mean(scale=1 / topo_rate)
        
        # get the next simulated tree (simplified tree at end of interval 0 of non-simple ts)
        tree1 = sts.at(ts.at(0).interval[1], sample_lists=True)
        
        # store type of first event: 0=no-change, 1=tree-change, 2=topo-change
        if tree0.kc_distance(tree1, lambda_=1):      # diff in blens only
            if tree0.kc_distance(tree1, lambda_=0):  # diff in topology only
                results[lidx, 7] = 2
                results[lidx, 4] = tree1.interval[0]
                results[lidx, 5] = tree1.interval[0]
            else:
                results[lidx, 7] = 1
                results[lidx, 4] = tree1.interval[0]
        else:
            results[lidx, 7] = 0

        # if waiting distances already found then continue to next locus
        if results[lidx, 5]:
            continue
        
        # iterate over trees from index 2 -> End to get waiting distances until the 
        # first observed tree and topo-change events from simulation.
        for idx in range(2, 999999):
            interval = ts.at_index(idx).interval
            tree1 = sts.at(interval[0], sample_lists=True)
                
            # if no difference in branch lengths then go to next idx
            if tree0.kc_distance(tree1, lambda_=1):     # diff in blens only
                if not results[lidx, 4]:
                    results[lidx, 4] = tree1.interval[0]
                if tree0.kc_distance(tree1, lambda_=0):  # diff in topology only
                    results[lidx, 5] = tree1.interval[0]
                    break
    return results

In [9]:
# example results array
sptree = get_sptree(8)
get_sim_and_analytical(sptree, nsamples=1, neff=100000, nsites=500_000, seed=123, smc=False)

array([[5.54764539e-01, 1.53804181e-01, 1.57655692e+02, 5.68656759e+02,
        1.03000000e+02, 2.37900000e+03, 5.71678234e+06, 1.00000000e+00],
       [5.52203657e-01, 2.80267923e-01, 1.86793057e+02, 3.68032873e+02,
        9.90000000e+01, 9.90000000e+01, 4.84741270e+06, 0.00000000e+00],
       [5.38887852e-01, 2.11229182e-01, 2.04351239e+02, 5.21340846e+02,
        1.11400000e+03, 1.11400000e+03, 4.54040231e+06, 0.00000000e+00],
       [5.46230438e-01, 1.19758179e-01, 2.23907949e+02, 1.02126917e+03,
        4.18000000e+02, 8.74000000e+02, 4.08812904e+06, 0.00000000e+00]])

## ----

## Distribute jobs to run in parallel

Also, to compare results of the Hudson and SMC' ancestry methods

In [43]:
def distribute_jobs(sptree: toytree.ToyTree, nsamples: int, outname: str="validation_full"):
    """Parallelize get_sim_and_analytical() function across Ne values and reps.    
    """
    # compare smc and full
    smc_results = np.zeros((NLOCI * NREPS, NEFF_VALS, 8))
    full_results = np.zeros((NLOCI * NREPS, NEFF_VALS, 8))

    # Ne values to test over
    nes = np.linspace(NEFF_MIN, NEFF_MAX, NEFF_VALS).astype(int)

    # run jobs in parallel to fill array
    rasyncs = {}
    with ProcessPoolExecutor(max_workers=50) as pool:

        # apply a different seed to each rep
        rng = np.random.default_rng(SEED)
        for rep in range(NREPS):
            seed = rng.integers(1e9)

            # apply same seed for each diff value of Ne
            for nidx, neff in enumerate(nes):

                # submit smc' jobs
                kwargs = {
                    "sptree": sptree,
                    "nsamples": nsamples,
                    "neff": neff, 
                    "nsites": int(550000 - neff), 
                    "seed": seed, 
                    "smc": True,
                }
                rasyncs[(nidx, 'smc', rep)] = pool.submit(get_sim_and_analytical, **kwargs)
                
                # submit full (non-smc') jobs
                kwargs = {
                    "sptree": sptree, 
                    "nsamples": nsamples,
                    "neff": neff, 
                    "nsites": int(550000 - neff), 
                    "seed": seed, 
                    "smc": False,
                }
                rasyncs[(nidx, 'full', rep)] = pool.submit(get_sim_and_analytical, **kwargs)

    # collect results into large res array
    for key, future in rasyncs.items():
        nidx, name, rep = key
        ival = slice(NLOCI * rep, NLOCI * (rep + 1))
        if name == 'smc':
            iresults = future.result()
            smc_results[ival, nidx, :] = iresults
        else:
            iresults = future.result()
            full_results[ival, nidx, :] = iresults
           
    np.save(f"./{outname}_{sptree.ntips}pops_full.npy", full_results)
    np.save(f"./{outname}_{sptree.ntips}pops_smcprime.npy", smc_results)

### RUN it

In [50]:
# 10K reps for each job, split into 100 parallelizable jobs
NLOCI = 100
NREPS = 100

In [51]:
sptree = get_sptree(1)
distribute_jobs(sptree, nsamples=8, outname="validation-final")

In [52]:
sptree = get_sptree(2)
distribute_jobs(sptree, nsamples=4, outname="validation-final")

In [53]:
sptree = get_sptree(8)
distribute_jobs(sptree, nsamples=1, outname="validation-final")

### Load results

In [143]:
def load_errors(full_path: str, smc_path: str):
    """Return the errors from the SMC' approximation.
    """
    res0 = np.load(full_path)  
    res1 = np.load(smc_path)  

    newdata = pd.DataFrame(
        data={
            'fullsim_minus_prediction_tree_error_perc': (
                (res0[:, :, 4] - res0[:, :, 2]) / 
                res0[:, :, 4]
            ).mean(axis=0),
            'fullsim_minus_prediction_tree_error_perc_sem': stats.sem(
                (res0[:, :, 4] - res0[:, :, 2]) / 
                res0[:, :, 4]
            ),
 
            'fullsim_minus_prediction_topo_error_perc': (
                (res0[:, :, 5] - res0[:, :, 3]) / 
                res0[:, :, 5]
            ).mean(axis=0),  
            'fullsim_minus_prediction_topo_error_perc_sem': stats.sem(
                (res0[:, :, 5] - res0[:, :, 3]) / 
                res0[:, :, 5]
            ),
            
            'smcsim_minus_prediction_tree_error_perc': (
                (res1[:, :, 4] - res1[:, :, 2]) / 
                res0[:, :, 4]
            ).mean(axis=0), 
            'smcsim_minus_prediction_tree_error_perc_sem': stats.sem(
                (res1[:, :, 4] - res1[:, :, 2]) / 
                res0[:, :, 4]
            ),
            
            'smcsim_minus_prediction_topo_error_perc': (
                (res1[:, :, 5] - res1[:, :, 3]) / 
                res0[:, :, 5]
            ).mean(axis=0), 
            'smcsim_minus_prediction_topo_error_perc_sem': stats.sem(
                (res1[:, :, 5] - res1[:, :, 3]) / 
                res0[:, :, 5]
            )
        },
        index=np.linspace(NEFF_MIN, NEFF_MAX, NEFF_VALS).astype(int),
    )
    return newdata

In [144]:
load_errors("./validation-final_8pops_full.npy", "./validation-final_8pops_smcprime.npy")

,fullsim_minus_prediction_tree_error_perc,fullsim_minus_prediction_tree_error_perc_sem,fullsim_minus_prediction_topo_error_perc,fullsim_minus_prediction_topo_error_perc_sem,smcsim_minus_prediction_tree_error_perc,smcsim_minus_prediction_tree_error_perc_sem,smcsim_minus_prediction_topo_error_perc,smcsim_minus_prediction_topo_error_perc_sem
50000,-4.590301,0.210143,-5.817518,0.611133,0.076681,0.235017,-0.667029,0.524612
100000,-4.402296,0.175215,-5.344043,0.306003,-0.176316,0.184483,0.075489,0.381169
150000,-4.131695,0.147960,-5.020287,0.241017,0.125853,0.164224,0.379740,0.274494
200000,-3.794021,0.132647,-4.434418,0.185454,0.061661,0.151745,0.115508,0.177933
250000,-3.920191,0.129899,-4.518322,0.177365,-0.125024,0.132088,0.078071,0.219767
300000,-3.455226,0.111627,-3.999984,0.146726,0.134410,0.140733,0.086524,0.172295
350000,-3.365110,0.105348,-3.866804,0.140165,-0.152067,0.114256,0.109440,0.179630
400000,-3.445599,0.106307,-4.013815,0.140698,0.000373,0.107967,0.278140,0.165505
450000,-3.252534,0.096555,-3.737175,0.128818,-0.168442,0.098568,0.273665,0.155026
500000,-3.266666,0.098897,-3.757325,0.132450,-0.079732,0.103310,-0.179083,0.129173


In [126]:
def load_data(path):
    """Return large array results summarized into a dataframe.
    
    # 0 = smc_tree_probs
    # 1 = smc_topo_probs
    # 2 = smc_tree_dists
    # 3 = smc_topo_dists
    # 4 = sim_tree_dists
    # 5 = sim_topo_dists
    # 6 = sum_edge_lengths of genealogy 0
    # 7 = event type
    """
    results = np.load(path)  
    
    newdata = pd.DataFrame(
        data={
            'tree_smc_prob': results[:, :, 0].mean(axis=0),
            'topo_smc_prob': results[:, :, 1].mean(axis=0),
            'tree_smc_dist': results[:, :, 2].mean(axis=0),
            'topo_smc_dist': results[:, :, 3].mean(axis=0),       
            'tree_sim_prob': (results[:, :, 7] != 0).mean(0),
            'topo_sim_prob': (results[:, :, 7] == 2).mean(0),
            'tree_sim_dist': results[:, :, 4].mean(axis=0),
            'topo_sim_dist': results[:, :, 5].mean(axis=0),

            'tree_smc_prob_CI95':  tuple(zip(
                np.percentile(results[:, :, 0], 2.5, axis=0).round(5),
                np.percentile(results[:, :, 0], 97.5, axis=0).round(5),
            )),
            'topo_smc_prob_CI95':  tuple(zip(
                np.percentile(results[:, :, 1], 2.5, axis=0).round(5),
                np.percentile(results[:, :, 1], 97.5, axis=0).round(5),
            )),

            'tree_smc_dist_CI95':  tuple(zip(
                np.percentile(results[:, :, 2], 2.5, axis=0).round(5),
                np.percentile(results[:, :, 2], 97.5, axis=0).round(5),
            )),
            'topo_smc_dist_CI95':  tuple(zip(
                np.percentile(results[:, :, 3], 2.5, axis=0).round(5),
                np.percentile(results[:, :, 3], 97.5, axis=0).round(5),
            )),
            'tmrca': results[:, :, 6].mean(axis=0),
        },
        index=np.linspace(NEFF_MIN, NEFF_MAX, NEFF_VALS).astype(int),
    )
    newdata['nsamples'] = NLOCI * NREPS
    return newdata

In [127]:
data = load_data("./validation_full_2pops.npy")
data

,tree_smc_prob,topo_smc_prob,tree_smc_dist,topo_smc_dist,tree_sim_prob,topo_sim_prob,tree_sim_dist,topo_sim_dist,tree_smc_prob_CI95,topo_smc_prob_CI95,tree_smc_dist_CI95,topo_smc_dist_CI95,tmrca,nsamples
50000,0.478757,0.111213,676.268799,3654.401925,0.4774,0.1093,675.3757,3582.1332,"(0.35682, 0.58968)","(0.03462, 0.23607)","(400.46634, 1094.71949)","(1130.48907, 9634.67349)",1.631340e+06,10000
100000,0.646151,0.225855,363.881016,1321.674949,0.6474,0.2282,362.5759,1303.2978,"(0.53602, 0.72513)","(0.07393, 0.45016)","(219.17838, 579.45744)","(411.92082, 3439.35414)",2.258646e+06,10000
150000,0.709904,0.313065,260.326753,701.879038,0.7104,0.3126,265.0879,717.6677,"(0.6106, 0.77856)","(0.11164, 0.51751)","(154.84618, 415.39722)","(285.81673, 1775.83932)",2.897600e+06,10000
200000,0.740947,0.367595,207.779292,471.085088,0.7430,0.3687,210.0198,475.3567,"(0.64008, 0.80942)","(0.14678, 0.56616)","(119.75943, 332.09229)","(216.09206, 1117.71679)",3.512331e+06,10000
250000,0.758430,0.399244,175.429929,365.796188,0.7595,0.4050,175.4391,364.2903,"(0.64994, 0.82855)","(0.16764, 0.59391)","(101.35483, 291.65201)","(179.65041, 839.06861)",4.090813e+06,10000
300000,0.770349,0.421542,152.285179,300.711882,0.7724,0.4228,154.2638,304.1593,"(0.65554, 0.84176)","(0.18132, 0.61643)","(86.30296, 252.68551)","(152.85254, 656.09956)",4.672879e+06,10000
350000,0.778406,0.434678,134.704801,257.274349,0.7810,0.4326,135.8442,259.2106,"(0.66137, 0.85099)","(0.19871, 0.63489)","(75.19905, 227.33365)","(133.13225, 514.27454)",5.245334e+06,10000
400000,0.783426,0.442923,120.886196,227.550116,0.7860,0.4406,121.9483,228.4511,"(0.65704, 0.85879)","(0.19749, 0.64864)","(66.14219, 206.37715)","(118.10011, 444.08066)",5.849824e+06,10000
450000,0.789172,0.450712,110.766089,204.429160,0.7896,0.4551,112.1042,209.1004,"(0.66305, 0.86532)","(0.20441, 0.65532)","(60.65237, 189.04474)","(109.50605, 387.53868)",6.347644e+06,10000
500000,0.791365,0.452936,101.172475,186.493868,0.7933,0.4496,101.1601,185.2469,"(0.6602, 0.86852)","(0.20559, 0.66422)","(54.35087, 175.71241)","(99.2669, 354.14556)",6.969181e+06,10000


# Plot results

### Plot probabilities

In [128]:
def plot_probs(data, topo: bool=False, smc: bool=False):
    """Plot probabilities of tree or topology change.
    
    """      
    smckey = "topo_smc_" if topo else "tree_smc_"
    simkey = "topo_sim_" if topo else "tree_sim_"
    color = toytree.color.COLORS2[0]
    canvas = toyplot.Canvas(width=350, height=300)
    
    # setup axes
    axb = canvas.cartesian(margin=65)
    axt = axb.share("y")
    axb.x.label.text = "N<sub>e</sub> (diploid)"
    axt.x.label.text = "Sptree edge lengths (coal units)"
    axb.y.label.text = "P(topo-change | S,G)" if topo else "P(tree-change | S,G)"
    axb.y.domain.max = 1
    axb.y.domain.min = 0
    axb.x.domain.min = 0
    
    # style axes
    for ax in (axb.x, axt.x, axb.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 7.5
        ax.ticks.far = 0
        ax.ticks.labels.offset = 12
        ax.ticks.labels.style["font-size"] = 16
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.ticks.style["stroke-width"] = 3
        ax.spine.style["stroke-width"] = 3
        
    avgdist = np.min([i.dist for i in sptree if not i.is_root()])
    axb.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(50_000, 500_000, 4),
        #np.linspace(0.5, 5, 5),
    )
    axt.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(50_000, 500_000, 4),
        (avgdist / (2 * np.linspace(50_000, 500_000, 4))).round(1),
    )
        
    # plot data
    axb.hlines([0, 0.5, 1.0], style={"stroke": "grey", "stroke-width": 2, "stroke-dasharray": "6,6", "stroke-opacity": 0.7})
    axb.fill(
       data.index,
       [i[0] for i in data[smckey + "prob_CI95"]],
       [i[1] for i in data[smckey + "prob_CI95"]],
       opacity=0.33,
    )
    axb.plot(data.index, data[smckey + "prob"], stroke_width=2, color=color)
    style = dict(opacity=0.8, color='black', mstyle={"stroke": "none"})
    marks = [
        axb.scatterplot(data.index, data[smckey + "prob"], size=12, color=color, opacity=0.7),
        axb.scatterplot(data.index, data[simkey + "prob"], size=6, marker='s', **style),
    ]
    return canvas

In [161]:
for nspecies in [1, 2, 8]:
    for smc in [True, False]:
        for topo in [True, False]:
            data = load_data(f"./validation_full_{nspecies}pops.npy")
            canvas = plot_probs(data, topo=topo, smc=smc)
            ttt = 'topo' if topo else 'tree'
            #toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/validation-probs-{nspecies}pops-{ttt}-final.svg")


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tc906dfbfeb7b4695b5c49209cf9b429f"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3b4a88fe371d4f65a5e8e48588a846c7"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tef7800a805e64e63b5b5e7c50eb5981b"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2480edc13bae4127be57cabb681f5ce2"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1219b65a8b864ea3a27a3fd0298df2e4"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

...

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t06b4a5f27efe46089a8517a3b6b2535f"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8e731bcf35e94e7685358d097a5fec38"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf5690d0937944c2e82355b2b23736340"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tcdbbe237a141453ebb6145203ce52ca0"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(topo-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ted4ec8545c2f4767b1877c9516d25d88"> 50000 200000 350000 500000 N e (diploid) 0.0 0.5 1.0 P(tree-change | S,G) 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

## Plot distances

In [130]:
def plot_distances(data, topo: bool=False, log: bool=True, smc: bool=False):
    """Plot waiting distances until tree or topology change.
    
    """
    smckey = "topo_smc_" if topo else "tree_smc_"
    simkey = "topo_sim_" if topo else "tree_sim_"
    color = toytree.color.COLORS2[0]
    canvas = toyplot.Canvas(width=350, height=300)
    
    # setup axes
    axb = canvas.cartesian(margin=65)
    axt = axb.share("y")#, yscale="log")
    axb.x.label.text = "N<sub>e</sub> (diploid)"
    axt.x.label.text = "Sptree edge lengths (coal units)"
    axb.y.label.text = "Distance to topo-change" if topo else "Distance to tree-change"
    axb.x.domain.min = 0
        
    if log:
        axt.y.scale = "log"
        axb.y.domain.min = 50
        axb.y.domain.max = 1e4
    else:
        axb.y.domain.min = 10
        axb.y.domain.max = 10000
    
    # style axes
    for ax in (axb.x, axt.x, axb.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 7.5
        ax.ticks.far = 0
        ax.ticks.labels.offset = 12
        ax.ticks.labels.style["font-size"] = 16
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.ticks.style["stroke-width"] = 3
        ax.spine.style["stroke-width"] = 3

    avgdist = np.min([i.dist for i in sptree if not i.is_root()])
    axb.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
    )
    axt.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
        (avgdist / (2 * np.linspace(NEFF_MIN, NEFF_MAX, 4))).round(1),
    )
    axt.y.ticks.locator = toyplot.locator.Explicit([100, 1000, 10000])

    axb.hlines([100, 1000, 10000], style={"stroke": "grey", "stroke-width": 2, "stroke-dasharray": "6,6", "stroke-opacity": 0.7})

    # plot data
    axb.fill(
       data.index,
       [i[0] for i in data[smckey + "dist_CI95"]],
       [i[1] for i in data[smckey + "dist_CI95"]],
       opacity=0.33,
    )
    axb.plot(data.index, data[smckey + "dist"], stroke_width=3, color=color)
    style = dict(opacity=0.8, color='black', mstyle={"stroke": "none"})
    marks = [
        axb.scatterplot(data.index, data[smckey + "dist"], size=12, marker="o", color=color, opacity=0.7),
        axb.scatterplot(data.index, data[simkey + "dist"], size=6, marker='s', **style),
    ]
    return canvas

In [159]:
for nspecies in [1, 2, 8]:
    for smc in [True, False]:
        for topo in [True, False]:
            data = load_data(f"./validation_full_{nspecies}pops.npy")
            canvas = plot_distances(data, topo=topo, log=True, smc=smc)
            ttt = 'topo' if topo else 'tree'
            #toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/validation-dists-{nspecies}pops-{ttt}-final.svg")


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t27f0597cf42f4c5d8cdc44667227ac9b"> <polygon points="87.0,130.9312400111314 109.0,153.1211890276889 131.0,165.90656930834396 153.0,175.01155916025917 175.0,182.11778798530318 197.0,188.5434000261394 219.0,192.98242127727983 241.0,196.9346741929826 263.0,200.90619559064405 285.0,204.21472979109384 285.0,150.86776748619724 263.0,147.98732819799466 241.0,144.06714426077997 219.0,140.22340087140742 197.0,134.98843089719196 175.0,128.65953792973346 153.0,121.02457705626742 131.0,113.10783550852958 109.0,99.96387508172043 87.0,77.4497199329749" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t907d85f467b3425f9a48d369cd8b7cea"> <polygon points="87.0,152.66254941923904 109.0,174.8902772615859 131.0,187.59941426596254 153.0,196.4760425545723 175.0,204.4658370253567 197.0,209.64942689970553 219.0,215.1222856218903 241.0,218.95731240042852 263.0,222.97752114111876 285.0,226.23510769297377 285.0,176.75398425401104 263.0,173.22901483655752 241.0,170.07597152638445 219.0,165.52805562879533 197.0,160.83017259732873 175.0,154.3044294955837 153.0,147.3100809107918 131.0,138.09532902420082 109.0,124.5553027573649 87.0,102.6179329047641" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6b7bfba2b54341c79be059cc5bc33671"> <polygon points="87.0,130.9312400111314 109.0,153.1211890276889 131.0,165.90656930834396 153.0,175.01155916025917 175.0,182.11778798530318 197.0,188.5434000261394 219.0,192.98242127727983 241.0,196.9346741929826 263.0,200.90619559064405 285.0,204.21472979109384 285.0,150.86776748619724 263.0,147.98732819799466 241.0,144.06714426077997 219.0,140.22340087140742 197.0,134.98843089719196 175.0,128.65953792973346 153.0,121.02457705626742 131.0,113.10783550852958 109.0,99.96387508172043 87.0,77.4497199329749" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="ta4b42b6106de4f8fa9bf570b7f948510"> <polygon points="87.0,152.66254941923904 109.0,174.8902772615859 131.0,187.59941426596254 153.0,196.4760425545723 175.0,204.4658370253567 197.0,209.64942689970553 219.0,215.1222856218903 241.0,218.95731240042852 263.0,222.97752114111876 285.0,226.23510769297377 285.0,176.75398425401104 263.0,173.22901483655752 241.0,170.07597152638445 219.0,165.52805562879533 197.0,160.83017259732873 175.0,154.3044294955837 153.0,147.3100809107918 131.0,138.09532902420082 109.0,124.5553027573649 87.0,102.6179329047641" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tcb123bdcd7804a438084e16209125537"> <polygon points="87.0,134.94464121772705 109.0,167.3375028021182 131.0,179.06418023276655 153.0,188.0368152069686 175.0,193.9627707597719 197.0,199.14586006828887 219.0,203.57787355452803 241.0,207.42206718596535 263.0,209.84623029060748 285.0,212.99600288597787 285.0,172.18638793917208 263.0,169.29522474182193 241.0,164.92546132414395 219.0,160.21685778345062 197.0,152.40222573059424 175.0,144.50979766852822 153.0,135.30920873939039 131.0,120.45402568527516 109.0,99.24506755968991 87.0,66.19412166091561" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

...

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5f5081d8993a4f71aabd959954f27f73"> <polygon points="87.0,168.2423641935958 109.0,187.58179571059867 131.0,198.73007577949568 153.0,206.97439762414044 175.0,212.32812880117845 197.0,217.48633574481687 219.0,221.90534259842687 241.0,226.022952204166 263.0,228.80311375742923 285.0,232.3228491931102 285.0,194.67392278462518 263.0,192.32733553664428 241.0,189.5127298303556 219.0,186.40961538736371 197.0,183.01729789513573 175.0,178.41571163332347 153.0,174.2493386323196 131.0,167.06785259931573 109.0,156.38791672331445 87.0,135.97626531480125" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6cf81f2147484388a4f35dc610358d0c"> <polygon points="87.0,156.3477982963669 109.0,182.60787808989554 131.0,193.73645876643482 153.0,200.8421867937689 175.0,206.39578158544677 197.0,210.32646444776938 219.0,213.644428496024 241.0,216.3005311865045 263.0,219.19998034608915 285.0,221.7441912277899 285.0,196.53428460220786 263.0,194.2505668013531 241.0,191.45797145470607 219.0,188.13068425634066 197.0,183.80065180630538 175.0,178.27860692147394 153.0,170.40097674240926 131.0,156.19772809331627 109.0,133.0867957970476 87.0,75.80938202018669" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t5f2b72eb15f04db19dce6a674a8b00ee"> <polygon points="87.0,185.23076998240208 109.0,201.1592170891338 131.0,210.1809007436805 153.0,216.94499990168035 175.0,221.78957253535242 197.0,225.62060540472453 219.0,229.35007365684942 241.0,232.3769879755382 263.0,235.00429801005026 285.0,235.07942741708544 285.0,212.3139180620835 263.0,210.53967929810344 241.0,208.7300779811655 219.0,206.67676488491145 197.0,204.60256680809263 175.0,201.81493919710138 153.0,198.3905745975864 131.0,193.9684168192156 109.0,187.1932350256918 87.0,172.08138398648202" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td613ccdf8d8145ee965f546bd10d66ee"> <polygon points="87.0,156.3477982963669 109.0,182.60787808989554 131.0,193.73645876643482 153.0,200.8421867937689 175.0,206.39578158544677 197.0,210.32646444776938 219.0,213.644428496024 241.0,216.3005311865045 263.0,219.19998034608915 285.0,221.7441912277899 285.0,196.53428460220786 263.0,194.2505668013531 241.0,191.45797145470607 219.0,188.13068425634066 197.0,183.80065180630538 175.0,178.27860692147394 153.0,170.40097674240926 131.0,156.19772809331627 109.0,133.0867957970476 87.0,75.80938202018669" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to topo-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2d1f631014d04583a2dbe269132b9695"> <polygon points="87.0,185.23076998240208 109.0,201.1592170891338 131.0,210.1809007436805 153.0,216.94499990168035 175.0,221.78957253535242 197.0,225.62060540472453 219.0,229.35007365684942 241.0,232.3769879755382 263.0,235.00429801005026 285.0,235.07942741708544 285.0,212.3139180620835 263.0,210.53967929810344 241.0,208.7300779811655 219.0,206.67676488491145 197.0,204.60256680809263 175.0,201.81493919710138 153.0,198.3905745975864 131.0,193.9684168192156 109.0,187.1932350256918 87.0,172.08138398648202" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) 100 1000 10000 Distance to tree-change 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

## Plot error

In [154]:
def plot_error(fullpath, smcpath, topo:bool=False):
    """Plot the percent error for waiting distances between the analytical expectation
    and result of stochastic simulation, when data were simulated under either the
    full coalescent with recombination, or the SMC'.    
    """
    
    fdata = load_errors(fullpath, smcpath)
    key = 'topo' if topo else 'tree'
    color = toytree.color.COLORS2[0]
    canvas = toyplot.Canvas(width=350, height=300)

    # setup axes
    axb = canvas.cartesian(margin=65)
    axt = axb.share("y")#, yscale="log")
    axb.x.label.text = "N<sub>e</sub> (diploid)"
    axt.x.label.text = "Sptree edge lengths (coal units)"
    axb.y.label.text = "Mean percent error"
    axb.x.domain.min = 0
    axb.y.domain.min = -10
    axb.y.domain.max = 10

    # style axes
    for ax in (axb.x, axt.x, axb.y):
        ax.domain.show = False
        ax.ticks.show = True
        ax.ticks.near = 7.5
        ax.ticks.far = 0
        ax.ticks.labels.offset = 12
        ax.ticks.labels.style["font-size"] = 16
        ax.label.offset = 35
        ax.label.style["font-size"] = 16
        ax.ticks.style["stroke-width"] = 3
        ax.spine.style["stroke-width"] = 3

    sptree = get_sptree(nspecies)
    avgdist = np.min([i.dist for i in sptree if not i.is_root()])
    axb.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
    )
    axt.x.ticks.locator = toyplot.locator.Explicit(
        np.linspace(NEFF_MIN, NEFF_MAX, 4),
        (avgdist / (2 * np.linspace(NEFF_MIN, NEFF_MAX, 4))).round(1),
    )

    axb.hlines([0], style={"stroke": "grey", "stroke-width": 2, "stroke-dasharray": "6,6", "stroke-opacity": 0.7})

    # plot data
    axb.fill(
       fdata.index,
       fdata[f"fullsim_minus_prediction_{key}_error_perc"] - fdata[f"fullsim_minus_prediction_{key}_error_perc_sem"], 
       fdata[f"fullsim_minus_prediction_{key}_error_perc"] + fdata[f"fullsim_minus_prediction_{key}_error_perc_sem"], 
       opacity=0.33,
    )
    axb.fill(
       data.index,
       fdata[f"smcsim_minus_prediction_{key}_error_perc"] - fdata[f"smcsim_minus_prediction_{key}_error_perc_sem"], 
       fdata[f"smcsim_minus_prediction_{key}_error_perc"] + fdata[f"smcsim_minus_prediction_{key}_error_perc_sem"], 
       opacity=0.33, color='black',
    )
    style = dict(opacity=0.8, color='black', mstyle={"stroke": "none"})
    marks = [
        axb.plot(fdata.index, fdata[f"fullsim_minus_prediction_{key}_error_perc"], 
                 size=6, marker="o", color=color, opacity=0.7),
        axb.plot(fdata.index, fdata[f"smcsim_minus_prediction_{key}_error_perc"], 
                 size=6, marker='s', **style),
    ]
    return canvas

In [164]:
nspecies = 8
for nspecies in [1, 2, 8]:
    for topo in [True, False]:
        canvas = plot_error(f"./validation_full_{nspecies}pops.npy", f"./validation_smc_{nspecies}pops.npy", topo)
        #toyplot.svg.render(canvas, f"../manuscript/figures/alternatives/error-{nspecies}-{topo}-final.svg")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1b7de696c93a4e079da053776e2b978c"> <polygon points="87.0,224.12214475873282 109.0,204.14895257030392 131.0,204.05575256671028 153.0,193.71812648419078 175.0,194.5819288657445 197.0,197.13131561819193 219.0,191.7454056318207 241.0,191.22172036630204 263.0,188.4198218561337 285.0,194.89879482842733 285.0,190.98780367195724 263.0,185.34921304109264 241.0,187.5909459612778 219.0,187.8799567643133 197.0,192.35231887963928 175.0,189.9513855623726 153.0,188.9813125283032 131.0,197.1582454979621 109.0,197.06051447190742 87.0,210.76828578563908" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,155.71136007147228 109.0,163.7542942388242 131.0,158.53045011498187 153.0,155.89731833987233 175.0,152.941965741466 197.0,156.24449744898462 219.0,155.95205572143428 241.0,155.04640445160993 263.0,155.46999141406084 285.0,156.37095793460196 285.0,152.56665268382196 263.0,152.81663785151628 241.0,150.4533530102994 219.0,151.7453710691444 197.0,151.3674002716113 175.0,147.85634274127898 153.0,150.71697067164118 131.0,145.38340400304634 109.0,158.1638348579991 87.0,141.40603093101217" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) -10 -5 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t537c291c43284e65b735e5afb7d61347"> <polygon points="87.0,214.32607468027402 109.0,197.91194483097308 131.0,193.04819525269838 153.0,189.03172843366156 175.0,191.56084192736154 197.0,193.63591099100495 219.0,187.38176552727737 241.0,186.99919393306098 263.0,183.6999211601488 285.0,186.85538461955738 285.0,184.24133697914252 263.0,181.44619273275242 241.0,184.26422664573278 219.0,184.36385413200213 197.0,190.03282262864136 175.0,187.84239586352535 153.0,185.73271752148864 131.0,188.7833504101551 109.0,192.66052577421078 87.0,204.39216039565423" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,160.1530765827263 109.0,157.0276046273516 131.0,156.4102390605305 153.0,153.70261801010818 175.0,152.79420602310245 197.0,152.98326343307562 219.0,153.73531423644005 241.0,152.26371469435747 263.0,153.87912633347253 285.0,153.21308096201227 285.0,150.42933084522855 263.0,151.4647657721772 241.0,149.49010940764896 219.0,150.16629765209137 197.0,148.52621334056826 175.0,148.77660010064943 153.0,149.8474970034992 131.0,152.59267712274982 109.0,152.00674580882276 87.0,149.59447517938287" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) -10 -5 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t2f34692d30f54d7b99b088f07b521f12"> <polygon points="87.0,215.67740994807303 109.0,214.91994919241984 131.0,192.06549257992748 153.0,196.8781403738618 175.0,196.04819468404014 197.0,194.82466825097913 219.0,191.49664488341105 241.0,187.71421181526898 263.0,187.93559879494234 285.0,184.39670916533973 285.0,181.80105113711818 263.0,185.01976227773503 241.0,184.73048571120032 219.0,187.83575963210382 197.0,190.45446214082344 175.0,190.43586891009767 153.0,192.15157031504916 131.0,187.90707935532797 109.0,205.66553558333527 87.0,205.21135258788627" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,169.07903304392673 109.0,164.28124014473033 131.0,154.2906034495432 153.0,148.70298383224795 175.0,152.6344338276608 197.0,151.1874164295104 219.0,151.396469474013 241.0,151.6200730083725 263.0,152.75072787018902 285.0,155.16363266635858 285.0,152.2459869042096 263.0,149.45203056207671 241.0,147.06020958967363 219.0,147.56773269046346 197.0,147.57526871481343 175.0,148.2919305969093 153.0,142.53224403031032 131.0,148.66725976252349 109.0,154.96229074541696 87.0,159.0113561054311" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) -10 -5 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tcbb403d754c640c88a3ed7676097384d"> <polygon points="87.0,199.8224626575146 109.0,196.8248292413478 131.0,187.61182728230165 153.0,187.87814604746308 175.0,187.80696760228892 197.0,185.51104315988218 219.0,184.91950165961222 241.0,183.1557286075621 263.0,183.09308023244603 285.0,180.5320837534232 285.0,178.56987715766388 263.0,180.9548206863091 241.0,180.95524853524424 219.0,182.41217386144427 197.0,182.89107872875417 175.0,184.91387928766747 153.0,184.85069799872218 131.0,184.54053144756313 109.0,192.47019775849245 87.0,194.67594270901324" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,150.3178291654242 109.0,154.1911673779978 131.0,148.4425952834149 153.0,150.50725267895461 175.0,148.3010874735469 197.0,150.83236690768464 219.0,149.89732480204862 241.0,151.37077908703284 263.0,150.77095556401906 285.0,153.26536005624075 285.0,151.00697487304907 263.0,148.25669572810438 241.0,148.90012865368413 219.0,147.24988022111046 197.0,148.06065098786345 175.0,144.2828897010951 153.0,147.08805555699286 131.0,143.86141936973712 109.0,150.30384253011007 87.0,143.48061933746" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) -10 -5 0 5 10 Mean percent error 5.0 1.2 0.7 0.5 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t195a61f1c6644273ad3ea15ff504d94e"> <polygon points="87.0,204.64353459509093 109.0,198.0253860151895 131.0,194.72107864275043 153.0,189.26891341006223 175.0,189.9133370793414 197.0,185.247038723473 219.0,184.0592403720642 241.0,185.31336183162546 263.0,182.8609458620455 285.0,183.06308566839766 285.0,180.81143584835172 263.0,180.67103661932998 241.0,182.92149325648325 219.0,181.67642856183517 197.0,182.7526933934907 175.0,186.89813839481883 153.0,186.11619308493692 131.0,190.623792208835 109.0,192.82334180654146 87.0,194.25426651297613" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,160.12894866633297 109.0,152.59827731531584 131.0,149.10540957310766 153.0,150.5306099045489 175.0,151.20442066073574 197.0,150.72904747026496 219.0,150.59661529703828 241.0,149.0426035741359 263.0,148.99156677445114 285.0,152.62018105230413 285.0,150.4242351097908 263.0,146.35613282601895 241.0,146.2290236737553 219.0,147.54290464717488 197.0,147.80003825317252 175.0,147.46837656250196 153.0,147.50575183734117 131.0,144.43900591083997 109.0,146.1184037181305 87.0,151.21054107909373" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) -10 -5 0 5 10 Mean percent error 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0355be036f7d45b7b6e4cd80e0c2034e"> <polygon points="87.0,190.80377533433153 109.0,188.9088470723321 131.0,186.37706320242654 153.0,183.37667834789548 175.0,184.42576269519475 197.0,180.31825544542892 219.0,179.49888980669442 241.0,180.19119870882867 263.0,178.46725473407005 285.0,178.6072838716376 285.0,176.92603903435926 263.0,176.8258272337832 241.0,178.38398446701183 219.0,177.707981610505 197.0,178.42059373056517 175.0,182.21748009267236 153.0,181.12167318956617 131.0,183.86174665271372 109.0,185.9301928946395 87.0,187.23134027702696" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:0.33;stroke:none"> <polygon points="87.0,151.34585403983826 109.0,153.06679430196394 131.0,150.32615212192417 153.0,150.76571588014576 175.0,152.1854491628817 197.0,150.05374114273573 219.0,152.2637434534291 241.0,150.91454994806926 263.0,152.26957995274688 285.0,151.55585632080005 285.0,149.79958927039252 263.0,150.59392765377441 241.0,149.07910307637607 219.0,150.32139484108762 197.0,147.66128752748054 175.0,149.9399616400105 153.0,148.18605302944331 131.0,147.5343470594678 109.0,149.9305817852179 87.0,147.35056492286083" style="fill:rgb(0%,0%,0%);fill-opacity:1.0;opacity:0.33;stroke:none"> 50000 200000 350000 500000 N e (diploid) -10 -5 0 5 10 Mean percent error 2.5 0.6 0.4 0.2 Sptree edge lengths (coal units)